In [1]:
from scraper.scraper import Scraper 
bot = Scraper()
bot.accept_cookies()



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [/Users/FKhan/.wdm/drivers/chromedriver/mac64/100.0.4896.60/chromedriver] found in cache


In [2]:
bot.keys_search('hoodies')

In [27]:
bot.next_button()

<selenium.webdriver.remote.webelement.WebElement (session="ed54465c95759096bde24fad89e7c477", element="37594e66-512d-4ea5-8bb9-de34473dc76b")>

In [184]:
container = bot.product_container()

In [185]:
from lib2to3.pgen2 import driver
from time import time
from typing_extensions import Self
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager


In [186]:
list_properties = container.find_elements(By.XPATH, './ul/li')
imageLinks = container.find_elements(By.XPATH, "./ul/li/span/a/picture")

In [187]:
link_list = []
for property in list_properties:
    link_list.append(property.find_element(By.TAG_NAME, 'a').get_attribute('href'))
imageNames = []
for element in imageLinks:
    imageNames.append(element.find_element(By.TAG_NAME, 'img').get_attribute("src"))    

In [120]:
from sqlalchemy import create_engine
DATABASE_TYPE = 'postgresql'
DBAPI = 'psycopg2'
ENDPOINT = 'dcp1.ckiqoi4xq1wx.eu-west-2.rds.amazonaws.com' 
USER = 'postgres'
PASSWORD = 'Hamburg98'
PORT = 5432
DATABASE = 'postgres'
engine = create_engine(f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASE}")
df = pd.read_sql('test_1', engine)
id_scraper = list(df['id'])

In [189]:
import time
import uuid
property_dict = {'link':[],
                 'price':[],
                 'name':[],
                 'id' :[],
                 'UUID':[],
                 'imagelinks_1':[],
                }


for link in link_list[0:50]:
    bot.driver.get(link)
    time.sleep(1)
    id = bot.driver.find_element(By.XPATH, '//span[@class="product-code"]')
    # property_dict['id'].append(id.text)
    if id in id_scraper:
        continue
    else:
        # bot.driver.get(link) 
        # time.sleep(1)
        # clicker = bot.driver.find_element(By.XPATH, '//i[@class="fa fa-angle-down"]').click()
        # bot.clicker()
        # time.sleep(2)
        property_dict['link'].append(link)
        price = bot.driver.find_element(By.XPATH, '//span[@data-e2e="product-price"]')
        property_dict['price'].append(price.text)
        name = bot.driver.find_element(By.XPATH, '//h1[@itemprop="name"]')
        property_dict['name'].append(name.text)
        # id = bot.driver.find_element(By.XPATH, '//span[@class="product-code"]')
        property_dict['id'].append(id.text)   
        property_dict['UUID'].append(uuid.uuid4())
        # imageLinks_1 = WebDriverWait(bot.driver, 6).until(EC.presence_of_element_located((By.TAG_NAME, 'img').get_attribute("src")))
        # # bot.driver.find_element(By.TAG_NAME, 'img').get_attribute("src")
        # property_dict['imagelinks_1'].append(imageLinks_1)
for image in imageNames[0:50]:
    bot.driver.get(image)
    time.sleep(1)
    property_dict['imagelinks_1'].append(image)        


        
    

In [190]:
property_dict

{'link': ['https://www.jdsports.co.uk/product/blue-nike-sportswear-club-t-shirt/16476236/',
  'https://www.jdsports.co.uk/product/blue-nike-tape-t-shirt/16481506/',
  'https://www.jdsports.co.uk/product/black-nike-air-max-short-sleeve-t-shirt/16268505/',
  'https://www.jdsports.co.uk/product/pink-nike-academy-essential-t-shirt/16479728/',
  'https://www.jdsports.co.uk/product/grey-nike-hybrid-t-shirt/16450245/',
  'https://www.jdsports.co.uk/product/white-nike-festival-futura-t-shirt/16474226/',
  'https://www.jdsports.co.uk/product/brown-nike-tape-t-shirt/16478970/',
  'https://www.jdsports.co.uk/product/black-nike-tape-t-shirt/16476626/',
  'https://www.jdsports.co.uk/product/black-nike-back-graphic-fade-t-shirt/16477282/',
  'https://www.jdsports.co.uk/product/blue-nike-back-graphic-fade-t-shirt/16476054/',
  'https://www.jdsports.co.uk/product/blue-nike-festival-futura-t-shirt/16476662/',
  'https://www.jdsports.co.uk/product/black-nike-surfing-turtle-t-shirt/16478043/',
  'https:/

In [195]:
import json
from uuid import UUID


class UUIDEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, UUID):
            return obj.hex
        return json.JSONEncoder.default(self, obj)
j = json.dumps(property_dict, cls=UUIDEncoder, indent=4)
f = open('Nike_t-shirts.json', 'w')
print(j, file=f)
f.close()

In [191]:
import pandas as pd 
x = pd.DataFrame(property_dict)

In [192]:
x

,link,price,name,id,UUID,imagelinks_1
0,https://www.jdsports.co.uk/product/blue-nike-s...,£18.00,Nike Sportswear Club T-Shirt,Product Code: 16476236/527202,659bda16-7d08-4905-9c68-2a4565747ba1,https://i8.amplience.net/t/jpl/jd_product_list...
1,https://www.jdsports.co.uk/product/blue-nike-t...,£28.00,Nike Tape T-Shirt,Product Code: 16481506/526981,b0f6fe47-4f4a-418e-8811-57641346649e,https://i8.amplience.net/t/jpl/jd_product_list...
2,https://www.jdsports.co.uk/product/black-nike-...,£30.00,Nike Air Max Short Sleeve T-Shirt,Product Code: 16268505/526659,1e9a6606-926d-4f88-a376-3a83b8476790,https://i8.amplience.net/t/jpl/jd_product_list...
3,https://www.jdsports.co.uk/product/pink-nike-a...,£20.00,Nike Academy Essential T-Shirt,Product Code: 16479728/527523,a6d0aaad-dee4-4515-b919-cb515e5ab74d,https://i8.amplience.net/t/jpl/jd_product_list...
4,https://www.jdsports.co.uk/product/grey-nike-h...,£28.00,Nike Hybrid T-Shirt,Product Code: 16450245/526788,9cac5c54-4c32-4ab8-ba05-e4803be30c78,https://i8.amplience.net/t/jpl/jd_product_list...
5,https://www.jdsports.co.uk/product/white-nike-...,£28.00,Nike Festival Futura T-Shirt,Product Code: 16474226/551621,77af747e-9ff8-4f3e-827d-2be1560c4813,https://i8.amplience.net/t/jpl/jd_product_list...
6,https://www.jdsports.co.uk/product/brown-nike-...,£28.00,Nike Tape T-Shirt,Product Code: 16478970/527182,107ffe23-dc01-44a9-9650-ca68a5f95f10,https://i8.amplience.net/t/jpl/jd_product_list...
7,https://www.jdsports.co.uk/product/black-nike-...,£28.00,Nike Tape T-Shirt,Product Code: 16476626/526992,8220c391-f42d-447f-86be-a8aec177bffa,https://i8.amplience.net/t/jpl/jd_product_list...
8,https://www.jdsports.co.uk/product/black-nike-...,£28.00,Nike Back Graphic Fade T-Shirt,Product Code: 16477282/527181,cd83cf57-d2f3-46d4-9d30-9c786b699292,https://i8.amplience.net/t/jpl/jd_product_list...
9,https://www.jdsports.co.uk/product/blue-nike-b...,£28.00,Nike Back Graphic Fade T-Shirt,Product Code: 16476054/527185,bc842767-44e2-4201-b043-7c70dba3cfa1,https://i8.amplience.net/t/jpl/jd_product_list...


In [7]:
import pandas as pd
df = pd.read_json('/Users/FKhan/Downloads/miniconda3/DCP_Project/raw_data/Nike caps/data.json')
y = pd.DataFrame(df)

In [176]:
from sqlalchemy import create_engine
DATABASE_TYPE = 'postgresql'
DBAPI = 'psycopg2'
ENDPOINT = 'dcp1.ckiqoi4xq1wx.eu-west-2.rds.amazonaws.com' 
USER = 'postgres'
PASSWORD = 'Hamburg98'
PORT = 5432
DATABASE = 'postgres'
engine = create_engine(f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASE}")
# df = pd.read_sql('test_1', engine)
# id_scraper = list(df['id'])

In [193]:
x.to_sql('Nike_t-shirts', con= engine, if_exists='append', index=False )

50

In [98]:
imageLinks = container.find_elements(By.XPATH, "./ul/li/span/a/picture")

In [99]:
imageNames = []
for element in imageLinks:
    imageNames.append(element.find_element(By.TAG_NAME, 'img').get_attribute("src"))

In [194]:
bot.multiple_image_2(list=imageNames[0:50], file_path='/Users/FKhan/Downloads/miniconda3/DCP_Project/raw_data/Nike_t-shirts/')

In [396]:
# this function will aid with downloading images using the specific image link (its another way of dowloading compared to the one below)
import urllib.request
def search_image():
        url = 'https://i8.amplience.net/t/jpl/jd_product_list?plu=jd_355655_plc&qlt=92&w=363&h=463&v=1&fmt=auto'
        opener = urllib.request.build_opener()
        opener.addheaders = [('User-Agent', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36')]
        urllib.request.install_opener(opener)
        urllib.request.urlretrieve(
        url, f'Nike_Hat_27.png')
        print(f'downloaded!')
try:
    search_image()
except: print('Thats not a valid url')

downloaded!


In [5]:
# This function saves images using the product link rather than image link and saves in chosen path
bot.image_download(url = 'https://www.jdsports.co.uk/product/adidas-originals-camo-baseball-cap/16239652/', file_name='adidas_cap_2', file_path='/Users/FKhan/Downloads/miniconda3/DCP_Project/raw_data/')

In [196]:
import boto3
s3_client = boto3.client('s3')

In [205]:
# To upload multple files
import glob
png_files = glob.glob('/Users/FKhan/Downloads/miniconda3/DCP_Project/raw_data/Nike_TS/*.png')

for filename in png_files:
    s3_client.upload_file(filename, 'dcp1', filename)

In [204]:
# To upload individual files
response = s3_client.upload_file('/Users/FKhan/Downloads/miniconda3/DCP_Project/raw_data/Nike_TS/Nike_TS_data.json', 'dcp1', 'Nike_TS_data.json')

In [49]:
# To download files from a bucket
import requests
url = 'https://dcp1.s3.eu-west-2.amazonaws.com/Nike+caps/Nike_Hat_023018_023018.png'

response = requests.get(url)
with open('blastoise.png', 'wb') as f:
    f.write(response.content)